In [ ]:
%load_ext autoreload
%autoreload 2


import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch
from waffles.np02_utils.PlotUtils import plot_grid, plot_detectors, genhist, runBasicWfAnaNP02, runBasicWfAnaNP02Updating
from waffles.coldboxVD.utils.spybuffer_reader import create_waveform_set_from_spybuffer
from waffles.input_output.hdf5_structured import load_structured_waveformset

from waffles.data_classes.CalibrationHistogram import CalibrationHistogram
from waffles.plotting.plot import plot_ChannelWsGrid, plot_CustomChannelGrid
from waffles.plotting.plot import plot_CalibrationHistogram
from waffles.utils.fit_peaks.fit_peaks import fit_peaks_of_CalibrationHistogram

In [ ]:
import h5py
import os

def check_hdf5_run(run_number, base_path="/eos/experiment/neutplatform/protodune/experiments/ColdBoxVD/November2025run/structured_hdf5s"):
    """
    Controlla che il file HDF5 del run sia presente e carica alcune informazioni.
    """
    # Path al run
    run_path = os.path.join(base_path, f"run{run_number}")
    
    # Trova tutti i file hdf5 nella cartella del run
    files = [f for f in os.listdir(run_path) if f.endswith(".hdf5")]
    
    if not files:
        print(f"Nessun file HDF5 trovato per il run {run_number} in {run_path}")
        return None
    
    # Per semplicità prendiamo il primo file
    file_path = os.path.join(run_path, files[0])
    print(f"Apro il file: {file_path}")
    
    # Apro l'HDF5
    with h5py.File(file_path, "r") as f:
        print("Struttura del file HDF5:")
        def print_name(name):
            print(f" - {name}")
        f.visit(print_name)
        
        # Stampa info sul dataset 'adcs'
        if 'adcs' in f:
            dset = f['adcs']
            print(f"\nNumero di waveform: {dset.shape[0]}")
            print(f"Lunghezza di una waveform: {dset.shape[1]}")
            print(f"Primo waveform (prime 10 samples): {dset[0,:10]}")
        else:
            print("\nDataset 'adcs' non trovato nel file HDF5.")
    
    print("\nFile caricato correttamente.")

# --- Esempio di uso ---
run_number = 410492  # puoi cambiare qui
check_hdf5_run(run_number)



In [ ]:
import h5py
import os
import matplotlib.pyplot as plt

def plot_first_waveforms(run_number, base_path="/eos/experiment/neutplatform/protodune/experiments/ColdBoxVD/November2025run/structured_hdf5s"):
    """
    Plotta la prima waveform di ciascun canale per un run.
    """
    # Path al run
    run_path = os.path.join(base_path, f"run{run_number}")
    
    # Trova file HDF5
    files = [f for f in os.listdir(run_path) if f.endswith(".hdf5")]
    if not files:
        print(f"Nessun file HDF5 trovato per il run {run_number}")
        return
    
    file_path = os.path.join(run_path, files[0])
    print(f"Apro il file: {file_path}")
    
    with h5py.File(file_path, "r") as f:
        adcs = f['adcs'][:]           # shape: (n_waveforms, n_samples)
        channels = f['channels'][:]   # shape: (n_waveforms,)
        
        # Trovo tutti i canali unici
        unique_channels = sorted(set(channels))
        
        plt.figure(figsize=(10,6))
        for ch in unique_channels:
            # Prendo la prima waveform per quel canale
            idx = (channels == ch).nonzero()[0][0]  # indice della prima occorrenza
            plt.plot(adcs[idx], label=f"Ch {ch}")
        
        plt.xlabel("Sample index")
        plt.ylabel("ADC")
        plt.title(f"Endpoint 801 - First waveform for each channel (Run {run_number})")
        plt.legend()
        plt.grid(True)
        plt.show()

# --- Uso ---
plot_first_waveforms(410492)


In [ ]:
wfset_full = None

channel_to_load = [ 28, 29 ]
led_values = [ 11 , 11]

for channel, volt in zip(channel_to_load, led_values):
    filepath_folder = f"/eos/experiment/neutplatform/protodune/experiments/ColdBoxVD/November2025run/spy_buffer/LED_scan/{volt}V/"
    filename=filepath_folder+f"channel_{channel}.dat"
    wfset_tmp = create_waveform_set_from_spybuffer(filename=filename, WFs=20000, length=1024, config_channel=channel)
    if wfset_full is None:
        wfset_full = wfset_tmp
    else:
        wfset_full.merge(wfset_tmp)

In [ ]:
from glob import glob
import copy
def open_processed(run, dettype, output_dir, channels = None, endpoints=None, nwaveforms=None, mergefiles = False, verbose=True):
    """
    Open the processed waveform set for a given run and detector type.
    """
    try: 
        wfset = load_structured_waveformset(
            f"{output_dir}/processed/run{run:0d}_{dettype}/processed_merged_run{run:06d}_structured_{dettype}.hdf5",
            max_to_load=nwaveforms,
            channels_filter=channels,
            endpoint_filter=endpoints
        )
    except:
        files = glob(f"{output_dir}/processed/run{run:06d}_{dettype}/processed_*_run{run:06d}_*_{dettype}.hdf5")
        if verbose:
            print("List of files found:")
            print(files)
        if not mergefiles or len(files)==1:
            files = files[0]
            wfset = load_structured_waveformset(files, max_to_load=nwaveforms, channels_filter=channels, endpoint_filter=endpoints, verbose=verbose)
        else: 
            wfset = load_structured_waveformset(files[0], max_to_load=nwaveforms, channels_filter= channels, endpoint_filter=endpoints, verbose=verbose)
            for f in files[1:]:
                tmpwf = load_structured_waveformset(f, max_to_load=nwaveforms, verbose=False)
                wfset.merge(copy.deepcopy(tmpwf))
    return wfset